# 13 NetCDF and `xarray`

In this lesson, we will get acquainted with a popuar format for working with multidimensional datasets called NetCDF and the Python package `xarray` which is based on NetCDF. 

`xarray` is a Python package that augments NumPy by adding 

# `xarray.DataArray`

The `xarray.DataArray` is the primary data structure of the xarray package. It is an n-dimensional array with labeled dimensions. We can think of an `xarray.DataArray` as representing a single variable in the NetCDF data format: it holds the variable’s values, dimensions, and attributes.

Apart from variables, dimensions, and attributes, `xarray` introduces one more piece of information to keep track of a dataset’s content: in `xarray` each dimension has at least one set of **coordinates**. A dimension’s coordinates indicate the dimension’s values. We can think of the coordinate’s values as the tick labels along a dimension.


In [5]:
# Import packages
import os
import numpy as np
import pandas as pd
import xarray as xr


### Variable values

The underlying data in the `xarray.DataArray` is a `numpy.array` that holds the variable values. 

In [7]:
# Values of a single variable at each point of the coords 
temp_data = np.array([np.zeros((5,5)),
                      np.ones((5,5)),
                      np.ones((5,5))*2]).astype(int)
temp_data

array([[[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]],

       [[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]],

       [[2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2]]])

### Dimensions and Coordinates

To specify the dimensions of our upcoming `xarray.DataArray`, we must examine how we've constructed the `numpy.array` holding the temperature data. 
The first dimension is time, the second is latitude, and longitude the third. 

From our exercises, we can also see that the coordinates (values of each dimension) are:

- time coordinates are 2022-09-01, 2022-09-02, 2022-09-03
- latitude coordinates are 70, 60, 50, 40, 30 (notice decreasing order)
- longitude coordinates are 60, 70, 80, 90, 100 (notice increasing order)

We add the dimensions as a tuple of strings and coordinates as a dictionary:

In [8]:
# Names of the dimensions in the required order
dims = ('time', 'lat', 'lon')

# Create coordinates to use for indexing along each dimension 
coords = {'time':pd.date_range("2022-09-01", "2022-09-03"),
          'lat':np.arange(70, 20, -10),
          'lon':np.arange(60, 110, 10)}  

In [9]:
coords

{'time': DatetimeIndex(['2022-09-01', '2022-09-02', '2022-09-03'], dtype='datetime64[ns]', freq='D'),
 'lat': array([70, 60, 50, 40, 30]),
 'lon': array([ 60,  70,  80,  90, 100])}

#### Attributes

Next, we add the attributes (metadata) for our temperature data as a dictionary:

In [10]:
# Attributes (metadata) of the data array 
attrs = { 'title':'temperature across weather stations',
          'standard_name':'air_temperature',
          'units':'degree_c'}

#### Putting it all together

Finally, we put all these pieces together (data, dimensions, coordinates, and attributes) to create an `xarray.DataArray`:

In [11]:
# Initialize xarray.DataArray
temp = xr.DataArray(data = temp_data, 
                    dims = dims,
                    coords = coords,
                    attrs = attrs)
temp

<xarray.DataArray (time: 3, lat: 5, lon: 5)>
array([[[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]],

       [[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]],

       [[2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2]]])
Coordinates:
  * time     (time) datetime64[ns] 2022-09-01 2022-09-02 2022-09-03
  * lat      (lat) int64 70 60 50 40 30
  * lon      (lon) int64 60 70 80 90 100
Attributes:
    title:          temperature across weather stations
    standard_name:  air_temperature
    units:          degree_c

We can also update the variable’s attributes after creating the object. 
Notice that each of the coordinates is also an `xarray.DataArray`, so we can add attributes to them.

In [12]:
# Update attributes
temp.attrs['description'] = 'Simple example of an xarray.DataArray'

# Add attributes to coordinates 
temp.time.attrs = {'description':'date of measurement'}

temp.lat.attrs['standard_name']= 'grid_latitude'
temp.lat.attrs['units'] = 'degree_N'

temp.lon.attrs['standard_name']= 'grid_longitude'
temp.lon.attrs['units'] = 'degree_E'
temp

<xarray.DataArray (time: 3, lat: 5, lon: 5)>
array([[[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]],

       [[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]],

       [[2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2],
        [2, 2, 2, 2, 2]]])
Coordinates:
  * time     (time) datetime64[ns] 2022-09-01 2022-09-02 2022-09-03
  * lat      (lat) int64 70 60 50 40 30
  * lon      (lon) int64 60 70 80 90 100
Attributes:
    title:          temperature across weather stations
    standard_name:  air_temperature
    units:          degree_c
    description:    Simple example of an xarray.DataArray

## Subsetting

At this point we have a single variable and the dataset attributes and variable attributes are the name.

To select data from an `xarray.Dataarray` weneed to specify the subsets we want along each dimension.

We can specify the data we need from each dimension by looking up the dimension by ots **position** or by looking up each dimension by its **name**.

## Example

We want to know what was the temperature recorded by the weather station located at 40°0′N 80°0′E on September 1st, 2022.

**Dimension lookup by position**

When we want to rely on the position of the dimensions in the `xarray.DataArray`, we need to remember that time is the first dimension, latitude is the second, and longitude the third.

Then, we can then access the values along each dimension in two ways:

- by integer: the exact same as a `numpy.array`. Use the locator brackets `[]` and use the integer location of the data you need to retrieve it:

In [13]:
# Access dimensions by position, then use integers for indexing
temp[0,3,2]

<xarray.DataArray ()>
array(0)
Coordinates:
    time     datetime64[ns] 2022-09-01
    lat      int64 40
    lon      int64 80
Attributes:
    title:          temperature across weather stations
    standard_name:  air_temperature
    units:          degree_c
    description:    Simple example of an xarray.DataArray

In [14]:
# Access dimensions by position, then use labels for indexing
temp.loc['2022-09-01', 40, 80]

<xarray.DataArray ()>
array(0)
Coordinates:
    time     datetime64[ns] 2022-09-01
    lat      int64 40
    lon      int64 80
Attributes:
    title:          temperature across weather stations
    standard_name:  air_temperature
    units:          degree_c
    description:    Simple example of an xarray.DataArray

For datasets with dozens of dimensions, it can be tough to remember which dimensions go where.

**Dimension lookup by name**

We can also use the dimension names to subset data, without the need to remember which dimensions goes where In this case, there are still two ways of selecting data along a dimension:

- by integer: we specify the integer location of the data we want along each dimension:

In [15]:
# Access dimensions by name, then use integers for indexing
temp.isel(time=0, lon=2, lat=3)

<xarray.DataArray ()>
array(0)
Coordinates:
    time     datetime64[ns] 2022-09-01
    lat      int64 40
    lon      int64 80
Attributes:
    title:          temperature across weather stations
    standard_name:  air_temperature
    units:          degree_c
    description:    Simple example of an xarray.DataArray

In [16]:
# Access dimensions by name, then use labels for indexing
temp.sel(time='2022-09-01', lat=40, lon=80)

<xarray.DataArray ()>
array(0)
Coordinates:
    time     datetime64[ns] 2022-09-01
    lat      int64 40
    lon      int64 80
Attributes:
    title:          temperature across weather stations
    standard_name:  air_temperature
    units:          degree_c
    description:    Simple example of an xarray.DataArray

Notice that the result of this indexing is a 1x1 `xarray.DataArray`. This is because operations on an `xarray.DataArray` always return another `xarray.DataArray`. In particular, operations returning scalar values will also produce `xarray` objects, so we need to cast them as numbers manually using the `xarray.DataArray` `item()` method:

In [17]:
temp.sel(time='2022-09-01', lat=40, lon=80).item()

0

## Reduction

`xarray` has implemented several methods to reduce an `xarray.DataArray` along any number of dimensions. For example, we can calculate the average temperature at each weather station over time and obtain a new `xarray.DataArray`.

In [18]:
avg_temp = temp.mean(dim='time')

In [19]:
avg_temp

<xarray.DataArray (lat: 5, lon: 5)>
array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])
Coordinates:
  * lat      (lat) int64 70 60 50 40 30
  * lon      (lon) int64 60 70 80 90 100

In [20]:
avg_temp.attrs = {'title': 'Average tempersture across three days'}

In [21]:
avg_temp

<xarray.DataArray (lat: 5, lon: 5)>
array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])
Coordinates:
  * lat      (lat) int64 70 60 50 40 30
  * lon      (lon) int64 60 70 80 90 100
Attributes:
    title:    Average tempersture across three days

## `xarray.DataSet`

An `xarray.DataSet` resembles an in-memory representation of a NetCDF file and consists of **multiple** variables (each being an `xarray.DataArray`), with dimensions, coordinates, and attributes, forming a self-describing dataset. Attributes can be specific to each variable, each dimension, or they can describe the whole dataset. The variables in an `xarray.DataSet` can have the same dimensions, share some dimensions, or have no dimensions in common.

Let's create a Dataset for the avg temp and temp data.

In [22]:
# Dictionary with variables
data_vars = {'temp':temp,
             'avg_temp':avg_temp}

attrs = {'title':'Temperature data at weather stations: daily and average',
         'description':'Simple example of an xarray.DataSet'}

# Create xarray.Dataset
temp_dataset = xr.Dataset( data_vars = data_vars,
                           attrs = attrs
                           )

In [23]:
temp_dataset

<xarray.Dataset>
Dimensions:   (time: 3, lat: 5, lon: 5)
Coordinates:
  * time      (time) datetime64[ns] 2022-09-01 2022-09-02 2022-09-03
  * lat       (lat) int64 70 60 50 40 30
  * lon       (lon) int64 60 70 80 90 100
Data variables:
    temp      (time, lat, lon) int64 0 0 0 0 0 0 0 0 0 0 ... 2 2 2 2 2 2 2 2 2 2
    avg_temp  (lat, lon) float64 1.0 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 1.0
Attributes:
    title:        Temperature data at weather stations: daily and average
    description:  Simple example of an xarray.DataSet